In [1]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh


import numpy as np

In [4]:
class camera:
    def __init__(self, index, mode = "dev"):
        self.cap = cv2.VideoCapture(index)
        self.face_mesh = mp_face_mesh.FaceMesh(
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5)     
        self.success, self.image = self.cap.read()
        self.results = self.face_mesh.process(self.image)
        self.loop = 0
        self.dir_vector = np.array([0.0,0.0,0.0])
        self.dir_state = 0
        self.mode = mode
        
    def camera_update(self):
        self.success, self.image = self.cap.read()
    
    def get_face_mesh_data(self):
        self.image.flags.writeable = False
        self.image = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)
        self.results = self.face_mesh.process(self.image)
    
    def draw_face_mesh_data(self):
        self.image.flags.writeable = True
        self.image = cv2.cvtColor(self.image, cv2.COLOR_RGB2BGR)
        if self.results.multi_face_landmarks:
            for face_landmarks in self.results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image=self.image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_tesselation_style())
                mp_drawing.draw_landmarks(
                    image=self.image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_contours_style())
                mp_drawing.draw_landmarks(
                    image=self.image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_IRISES,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_iris_connections_style())
                # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Hands', cv2.flip(self.image, 1))
    
    def get_data(self):
        for face_landmarks in self.results.multi_face_landmarks:
            result = face_landmarks.landmark
        return result
    
    def release(self):
        self.cap.release()
    
    def calculate_face_dir_vector(self,data):
        # 143 : right eye end , 272 : left eye end , 199 : jaw end
        if(self.mode == "dev"):
            print("143 : ",[data[143].x,data[143].y,data[143].z]," 272: ",[data[272].x,data[272].y,data[272].z]," 199 : ",[data[199].x,data[199].y,data[199].z])
        vector_a = np.array([data[199].x,data[199].y,data[199].z]) - np.array([data[143].x,data[143].y,data[143].z])                          
        vector_b = np.array([data[272].x,data[272].y,data[272].z]) - np.array([data[199].x,data[199].y,data[199].z])
        if(self.mode == "dev"):
            print("vector a : ",vector_a," vector b : ",vector_b)
        result = np.cross(vector_a,vector_b) 
        if(self.mode == "dev"):
            print("origin result vector : ",result)
        return result / np.linalg.norm(result)
    
    def current_face_dir_state(self):
#         7,8,9
#         4,5,6       no face : -1
#         1,2,3
        if(self.dir_vector[0] == 0 and self.dir_vector[1] == 0 and self.dir_vector[2] == 0):
            self.dir_state = -1
            return
        self.dir_state = 5
        if(self.dir_vector[1] < -0.4):
            self.dir_state += 3
        elif(self.dir_vector[1] > 0.4):
            self.dir_state -= 3
        
        if(self.dir_vector[0] < -0.5):
            self.dir_state += 1
        elif(self.dir_vector[1] > 0.5):
            self.dir_state -= 1
    
    def current_face_dir_to_text(self):
        res = ""
        if(self.dir_state == -1):
            return "no face"
        
        if(self.dir_state % 3 == 0):
            res += "right "
        elif(self.dir_state % 1 == 1):
            res += "left "
            
        if(self.dir_state > 6):
            res += "up "
        elif(self.dir_state < 4):
            res += "down "
        return res
        
        
        
    def run(self):
        while self.cap.isOpened():
            self.camera_update()
            if not self.success:
                print("Ignoring empty camera frame.")
                # If loading a video, use 'break' instead of 'continue'.
                continue
            
            self.get_face_mesh_data()
            
            self.draw_face_mesh_data()
            
            
            if(self.results.multi_face_landmarks == None):
                self.dir_vector = np.array([0,0,0])
                if(self.mode != "build"):
                    print("no face")
                continue
                
            
            print(self.loop)
            data = self.get_data()
            self.dir_vector = self.calculate_face_dir_vector(data)
            self.current_face_dir_state()

            if(self.mode != "build"):
                print("direction : ",self.dir_state," ",self.current_face_dir_to_text())
                
            if(self.mode == "dev"):
                print(self.dir_vector)
                
            self.loop+=1
            if cv2.waitKey(5) & 0xFF == 27:
                break
        
        self.release()

In [ ]:
camera1 = camera(0, "only dir")
camera1.run()


# direction [0,0,-1] -> face front
# direction [-1,0,0] -> face right
# direction [1,0,0] -> face left
# direction [0,-1,0] -> face up
# direction [0,1,0] -> face down



no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
no face
0
direction :  8   up 
1
direction :  8   up 
2
direction :  8   up 
3
direction :  8   up 
4
direction :  8   up 
5
direction :  8   up 
6
direction :  8   up 
7
direction :  8   up 
8
direction :  8   up 
no face
9
direction :  8   up 
10
direction :  8   up 
11
direction :  8   up 
12
direction :  8   up 
13
direction :  8   up 
14
direction :  8   up 
15
direction :  8   up 
16
direction :  8   up 
17
direction :  8   up 
18
direction :  8   up 
19
direction :  8   up 
20
direction :  8   up 
21
direction :  8   up 
22
direction :  8   up 
23
direction :  8   up 
24
direction :  8   up 
25
direction :  5   
26
direction :  5   
27
direction :  5   
28
direction :  5   
29
direction :  5   
30
direction :  5   
31
direction :  5   
32
direction :  5   
33
direction :  5   
34
direction :  5   
35
direction :  5   
36
direction :  5   
37
direction :  5   
3

368
direction :  5   
369
direction :  5   
370
direction :  5   
371
direction :  5   
372
direction :  5   
373
direction :  5   
374
direction :  5   
375
direction :  5   
376
direction :  5   
377
direction :  5   
378
direction :  5   
379
direction :  5   
380
direction :  5   
381
direction :  5   
382
direction :  5   
383
direction :  5   
384
direction :  5   
385
direction :  5   
386
direction :  5   
387
direction :  5   
388
direction :  5   
389
direction :  5   
390
direction :  5   
391
direction :  5   
392
direction :  5   
393
direction :  5   
394
direction :  5   
395
direction :  5   
396
direction :  5   
397
direction :  5   
398
direction :  5   
399
direction :  5   
400
direction :  5   
401
direction :  5   
402
direction :  5   
403
direction :  5   
404
direction :  5   
405
direction :  5   
406
direction :  5   
407
direction :  5   
408
direction :  5   
409
direction :  5   
410
direction :  5   
411
direction :  5   
412
direction :  5   
413
direct

676
direction :  5   
677
direction :  5   
678
direction :  5   
679
direction :  5   
680
direction :  5   
681
direction :  5   
682
direction :  5   
683
direction :  5   
684
direction :  5   
685
direction :  5   
686
direction :  5   
687
direction :  5   
688
direction :  5   
689
direction :  5   
690
direction :  5   
no face
691
direction :  5   
no face
692
direction :  5   
no face
no face
no face
693
direction :  5   
no face
694
direction :  5   
no face
695
direction :  5   
no face
no face
no face
no face
696
direction :  5   
no face
697
direction :  5   
no face
no face
no face
698
direction :  5   
no face
699
direction :  5   
no face
no face
700
direction :  5   
no face
701
direction :  5   
no face
702
direction :  5   
no face
no face
703
direction :  5   
704
direction :  5   
no face
705
direction :  5   
706
direction :  5   
no face
no face
707
direction :  5   
no face
708
direction :  5   
709
direction :  5   
no face
no face
710
direction :  5   
711
di

992
direction :  5   
993
direction :  5   
994
direction :  5   
995
direction :  5   
996
direction :  5   
997
direction :  5   
998
direction :  5   
999
direction :  5   
1000
direction :  5   
1001
direction :  5   
1002
direction :  5   
1003
direction :  5   
1004
direction :  5   
1005
direction :  5   
1006
direction :  5   
1007
direction :  5   
1008
direction :  5   
1009
direction :  5   
1010
direction :  5   
1011
direction :  5   
1012
direction :  5   
1013
direction :  5   
1014
direction :  5   
1015
direction :  5   
1016
direction :  5   
1017
direction :  5   
1018
direction :  5   
1019
direction :  5   
1020
direction :  5   
1021
direction :  5   
1022
direction :  5   
1023
direction :  5   
1024
direction :  5   
1025
direction :  5   
1026
direction :  5   
1027
direction :  5   
1028
direction :  5   
1029
direction :  5   
1030
direction :  5   
1031
direction :  5   
1032
direction :  5   
1033
direction :  5   
1034
direction :  5   
1035
direction :  5

1341
direction :  6   right 
1342
direction :  6   right 
1343
direction :  6   right 
1344
direction :  6   right 
1345
direction :  5   
1346
direction :  5   
1347
direction :  5   
1348
direction :  5   
1349
direction :  5   
1350
direction :  5   
1351
direction :  5   
1352
direction :  5   
1353
direction :  5   
1354
direction :  5   
1355
direction :  5   
1356
direction :  5   
1357
direction :  5   
1358
direction :  5   
1359
direction :  5   
1360
direction :  5   
1361
direction :  5   
1362
direction :  5   
1363
direction :  5   
1364
direction :  5   
1365
direction :  8   up 
1366
direction :  8   up 
1367
direction :  8   up 
1368
direction :  8   up 
1369
direction :  8   up 
1370
direction :  8   up 
1371
direction :  8   up 
1372
direction :  8   up 
1373
direction :  8   up 
1374
direction :  8   up 
1375
direction :  8   up 
1376
direction :  8   up 
1377
direction :  8   up 
1378
direction :  8   up 
1379
direction :  8   up 
1380
direction :  8   up 
1381
dir